## <b><font color='darkblue'>Preface</font></b>
([source](https://learn.deeplearning.ai/courses/langchain/lesson/5/question-and-answer)) <b><font size='3ptx'>In the age of information overload, extracting precise answers from vast amounts of text is a critical challenge</font></b>.

<b>LangChain's Question Answering over Documents framework addresses this head-on, providing a powerful solution that leverages cutting-edge language models to streamline knowledge retrieval</b>. Whether you're a researcher, analyst, or developer, this technology has the potential to revolutionize your workflow.

### <b><font color='darkgreen'>Outline</font></b>
LangChain: Question Answering Over Documents – Key Concepts:
* **Document Retrieval Methods:** Leverage Large Language Models (LLMs) to answer questions extracted from PDFs, web pages, or internal documents, expanding the scope of LLM applications. This includes demonstrating simple indexing and utilizing RetrievalQA for querying. The latter is an abstraction of [Retrieval-Augmented Generation](https://en.wikipedia.org/wiki/Prompt_engineering#Retrieval-augmented_generation) (RAG), enhancing retrieval capabilities.
* **Data and Model Integration:** Combine LLMs with unstructured data using LangChain components like embedding models and vector stores, increasing model flexibility and adaptability.
* **Document Processing Techniques**: Utilize "[**CSVLoader**](https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.csv_loader.CSVLoader.html)" and "[**DocArrayInMemorySearch**](https://api.python.langchain.com/en/latest/vectorstores/langchain_community.vectorstores.docarray.in_memory.DocArrayInMemorySearch.html)" vector stores for document loading and indexing. Employ embedding techniques to convert text into numerical representations for efficient processing.

Additionally, four post-retrieval processing methods are briefly touched:
* **Stuff**: Combine all retrieved documents into a single input for the LLM.
* **Map_reduce**: Process each document individually and then aggregate the results.
* **Refine**: Iteratively refine the query based on previous responses.
* **Map_rerank**: Rerank documents based on their relevance to the refined query.

In [95]:
!pip freeze | grep -P '(openai|langchain)'

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


langchain==0.2.1
langchain-anthropic==0.1.15
langchain-community==0.0.38
langchain-core==0.2.3
langchain-google-genai==1.0.6
langchain-groq==0.1.3
langchain-openai==0.1.9
langchain-text-splitters==0.2.0
langchainhub==0.1.14
openai==1.28.1


In [1]:
import datetime
import os
import openai
from dotenv import load_dotenv, find_dotenv
from langchain_openai import ChatOpenAI
import pandas as pd


TEST_DATA = pd.DataFrame({'index': ['row1', 'row2'], 'review': ['review1', 'review2']})

a = load_dotenv(find_dotenv(os.path.expanduser('~/.env'))) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

## <b><font color='darkblue'>[CSVLoader](https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.csv_loader.CSVLoader.html): Load a CSV file into a list of Documents.</font></b>
<b><font size='3ptx'>Load csv data with a single row per document.</font></b>

In [68]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores.docarray.in_memory import DocArrayInMemorySearch
from langchain.indexes import VectorstoreIndexCreator
from IPython.display import display, Markdown
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain.embeddings import HuggingFaceEmbeddings

TEST_CSV_FILE_PATH = 'test_data/test_data.csv'

In [29]:
loader = CSVLoader(TEST_CSV_FILE_PATH)

In [30]:
print(loader.load()[0].page_content[:100])

: 0
name: Women's Campside Oxfords
description: This ultracomfortable lace-to-toe Oxford boasts a su


In [31]:
model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2" # Selecting a sentence embedding model
#model_kwargs = {'device': 'cuda'}
#encode_kwargs = {'normalize_embeddings': False}
hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
)

In [32]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=hf_embeddings,
).from_loaders([loader])

In [33]:
query ="Please recommend me product of car."

In [36]:
llm_replacement_model = OpenAI(temperature=0, 
                               model='gpt-3.5-turbo-instruct')

In [37]:
response = index.query(query, 
                       llm = llm_replacement_model)

In [38]:
display(Markdown(response))

 I'm sorry, I cannot recommend products as I am a text-based AI and do not have the ability to browse or access information about specific products. 

### <b><font color='darkgreen'>Step By Step</font></b>

In [53]:
loader = CSVLoader(file_path=TEST_CSV_FILE_PATH)
docs = loader.load()
docs[0]

Document(page_content=': 0\nname: car\ndescription: Experience the pinnacle of automotive craftsmanship with the "Luxury and Elegance". Immerse yourself in a world of refined luxury, where every detail – from the hand-stitched leather seats to the precision-engineered engine – exudes sophistication and performance. The "Luxury and Elegance" isn\'t just a car, it\'s a statement.', metadata={'source': 'test_data/test_data.csv', 'row': 0})

In [54]:
embed = hf_embeddings.embed_query("Hi my name is Harrison")

In [55]:
print(len(embed))

768


In [56]:
print(embed[:5])

[-0.12968699634075165, -0.09362716227769852, -0.0035989920143038034, 0.015314917080104351, 0.19877466559410095]


In [57]:
db = DocArrayInMemorySearch.from_documents(
    docs, 
    hf_embeddings
)

In [62]:
query_car ="Please recommend me product of car."

In [59]:
docs = db.similarity_search(query_car)

In [60]:
len(docs)

4

print(docs[0].page_content)

In [63]:
query_phone ="Please recommend me the product of phone."

In [64]:
docs = db.similarity_search(query_phone)

In [65]:
print(docs[0].page_content)

: 4
name: phone
description: Experience the future in your hands with the "Cutting-Edge Technology". This revolutionary device redefines what a smartphone can do. Its [Unique Feature 1] pushes boundaries, while [Unique Feature 2] transforms how you interact with your world. The "Cutting-Edge Technology" isn't just a phone; it's a portal to endless possibilities.


### <b><font color='darkgreen'>DB as retriever</font></b>

In [66]:
retriever = db.as_retriever()

In [69]:
llm = ChatOpenAI(temperature = 0.0, model=llm_model)

In [79]:
docs = loader.load()
qdocs = "".join([docs[i].page_content for i in range(len(docs))])

In [80]:
print(qdocs[:500])

: 0
name: car
description: Experience the pinnacle of automotive craftsmanship with the "Luxury and Elegance". Immerse yourself in a world of refined luxury, where every detail – from the hand-stitched leather seats to the precision-engineered engine – exudes sophistication and performance. The "Luxury and Elegance" isn't just a car, it's a statement.: 1
name: car
description: Unleash your inner explorer with the "Adventure and Thrills". Designed to conquer any terrain, this rugged and capable v


In [85]:
response = llm.invoke(f"{qdocs} Question: Please list all your products as car in a table in markdown and summarize each one.") 

In [87]:
display(Markdown(response.content))

| Name              | Summary                                                                                                      |
|-------------------|--------------------------------------------------------------------------------------------------------------|
| Luxury and Elegance | A pinnacle of automotive craftsmanship with refined luxury and sophistication.                                |
| Adventure and Thrills | A rugged and capable vehicle designed for exploration and unforgettable journeys.                            |
| Spark 3            | A futuristic driving experience with cutting-edge technology and advanced features.                           |
| Family and Practicality | A family-friendly vehicle with spacious seating, versatile cargo options, and a focus on safety.             |

## <b><font color='darkblue'>RetrievalQA</font></b>

In [88]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [89]:
query =  "Please list all your phone products in a table in markdown and summarize each one."

In [91]:
response = qa_stuff.invoke(query)



> Entering new RetrievalQA chain...

> Finished chain.


In [94]:
display(Markdown(response['result']))

| Name | Description |
|------|-------------|
| Cutting-Edge Technology | Experience the future in your hands with this revolutionary device that redefines what a smartphone can do. Its unique features push boundaries and transform how you interact with the world. It's a portal to endless possibilities. |
| Husky | Capture life's moments in stunning detail with the "Husky". Its advanced camera system empowers you to unleash your inner photographer, delivering professional-grade results for portraits, landscapes, and videos. |
| Ak3 | Make a statement with the "Ak3", a masterpiece of design and engineering. Its sleek lines and premium materials turn heads, while its unique design feature sets it apart as a fashion accessory that elevates your style. |

## <b><font color='darkblue'>RetrievalQA Chain Type</font></b>
<b><font size='3ptx'>Chain Types: Strategies for Answering</font></b>

![chain type](images/ch5_qa_chain_type.PNG)

### <b><font color='darkgreen'>stuff</font></b>
This is the simplest approach. All retrieved documents are "stuffed" into the LLM's prompt at once. The LLM then crafts an answer based on everything it sees.
* **Pros:** Easy to set up.
* **Cons:** Can struggle with longer documents or many documents, as LLMs have limited context windows.

![stuff](images/ch5_qa_chain_type_stuff.PNG)

### <b><font color='darkgreen'>Map_reduce</font></b>
This is a more sophisticated strategy:
* **Map:** The LLM processes each retrieved document separately, creating an initial answer for each.
* **Reduce:** The LLM combines these initial answers into a final, consolidated response.

It has below pros & cons:
* **Pros:** Handles longer documents and larger sets of documents more effectively.
* **Cons:** Slightly more complex to implement.

![map_reduce](images/ch5_qa_chain_type_map_reduce.PNG)

### <b><font color='darkgreen'>Refine</font></b>
This is similar to `map_reduce` but takes an iterative approach:
1. The LLM answers the question based on the first document.
2. It refines this initial answer using the second document.
3. This process continues until all documents are used.

It has below pros and cons:
* **Pros:** Can lead to more nuanced answers.
* **Cons:** Might be slower for large numbers of documents.

![refine](images/ch5_qa_chain_type_refine.PNG)

### <b><font color='darkgreen'>Map_rerank</font></b>
Similar to `map_reduce`, but it also re-ranks the documents based on how relevant they are to the initial answers.

* **Pros**: Utilizes a scoring mechanism to select the best answer, making it suitable for scenarios where the optimal choice needs to be made from multiple possible answers.
* **Cons**: Relies on the accuracy of the scoring mechanism, which might not be suitable for all types of documents or queries. The scoring model requires additional training, and the scoring metric itself involves strong subjective judgment, making it difficult to standardize.

![rerank](images/ch5_qa_chain_type_map_rerank.PNG)

## <b><font color='darkblue'>Supplement</font></b>
* [Deeplearning.ai - Langchain Ch2: Model, prompt and parser](https://github.com/johnklee/ml_articles/blob/master/deeplearning_ai/langchain/ch2_model_prompt_and_parser.ipynb)
* [Deeplearning.ai - Langchain Ch3: Memory](https://github.com/johnklee/ml_articles/blob/master/deeplearning_ai/langchain/ch3_memory.ipynb)
* [Deeplearning.ai - Langchain Ch4: Chain](https://github.com/johnklee/ml_articles/blob/master/deeplearning_ai/langchain/ch4_chains.ipynb)
* [Deeplearning.ai - Langchain Ch5: Question and answer](https://github.com/johnklee/ml_articles/blob/master/deeplearning_ai/langchain/ch5_question_and_answer.ipynb)
* [Deeplearning.ai - Langchain Ch6: Evaluation](https://github.com/johnklee/ml_articles/blob/master/deeplearning_ai/langchain/ch6_evaluation.ipynb)
* [Deeplearning.ai - Langchain Ch7: Agents](https://github.com/johnklee/ml_articles/blob/master/deeplearning_ai/langchain/ch7_agents.ipynb)
* [Medium - 4 Ways to Do Question Answering in LangChain](https://towardsdatascience.com/4-ways-of-question-answering-in-langchain-188c6707cc5a)
> LangChain makes it easy for you to do question answering with your documents. But do you know that there are at least 4 ways to do question answering in LangChain? In this blog post, we are going to explore four different ways to do question-answering and the various options you could consider for your use cases.

* [課程筆記: LangChain - Question and Answer over Documents](https://hackmd.io/@YungHuiHsu/BJ10qunzp)
* [LangChain-for-LLM-Application-Development/OutdoorClothingCatalog_1000.csv](https://github.com/Ryota-Kawamura/LangChain-for-LLM-Application-Development/blob/main/OutdoorClothingCatalog_1000.csv)
* [Pinecone Docs - Build a RAG chatbot](https://docs.pinecone.io/guides/get-started/build-a-rag-chatbot)